* Source :  https://medium.com/@arvinakori/opencv-brief-note-on-how-you-can-access-webcam-in-google-colab-d4d84efc301f


In [1]:
!git clone https://github.com/opencv/opencv.git
!mkdir Video


Cloning into 'opencv'...
remote: Enumerating objects: 299061, done.
remote: Total 299061 (delta 0), reused 0 (delta 0), pack-reused 299061
Receiving objects: 100% (299061/299061), 489.08 MiB | 24.92 MiB/s, done.
Resolving deltas: 100% (207972/207972), done.
Checking out files: 100% (6911/6911), done.


In [5]:

!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file_test = '/content/Video/osy_test.mp4' 
  

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");

my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Saving the recording... Please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;

    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
    


def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary

In [8]:
start_webcam()
videofile = get_video()
with open(video_file_test, 'wb') as f:
  f.write(videofile)


<IPython.core.display.Javascript object>

In [9]:
!ls /content/Video/osy_test.mp4

/content/Video/osy_test.mp4


In [10]:
import cv2

cam = cv2.VideoCapture('/content/Video/osy_test.mp4')
currentFrame = 0
while(True):
    ret, frame = cam.read()
    if ret:
        cv2.imwrite(str(currentFrame) + '.jpg', frame)
        currentFrame += 1
    else:
        break
cam.release()

In [11]:
import cv2  
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier('/content/opencv/data/haarcascades/haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('/content/opencv/data/haarcascades/haarcascade_eye.xml')  

cap = cv2.VideoCapture(video_file_test) 
  
count = 1
while cap.isOpened() & count<30:
  
    ret, img = cap.read()  
  
   # gray = cv2.cvtColor(img, cv2.COLOR_BGR2BGR) 
  
    # Detects faces of different sizes in the input image 
    faces = face_cascade.detectMultiScale(img, 1.3, 5) 
  
    for (x,y,w,h) in faces: 
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)  
       # roi_gray = gray[y:y+h, x:x+w] 
        roi_color = img[y:y+h, x:x+w] 
  
        eyes = eye_cascade.detectMultiScale(roi_color)  
  
        for (ex,ey,ew,eh) in eyes: 
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,127,255),2) 

    cv2_imshow(img) 
  
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
  
cap.release() 
cv2.destroyAllWindows()  

Output hidden; open in https://colab.research.google.com to view.